In [1]:
import pandas as pd

### Getting Data From https://raw.githubusercontent.com/CSSEGISandData/COVID-19

In [2]:
original_confirmed=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',error_bad_lines=False)
original_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',error_bad_lines=False)
original_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',error_bad_lines=False)

### Creating and Merging DataFrame


In [3]:
# Get dates in the column
dates = original_confirmed.columns[4:]

original_confirmed_tran = original_confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

original_deaths_trans = original_deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

original_recovered_trans = original_recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

# total final records
print(original_confirmed_tran.shape)

(26136, 6)


In [4]:
df_final = pd.merge(left=original_confirmed_tran, right=original_deaths_trans, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
df_final = pd.merge(left=df_final, right=original_recovered_trans, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

df_final.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0.0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0


In [5]:
# Delete Province/State as not analysed at this level
df_final.drop('Province/State',axis=1,inplace=True)

In [6]:
df_final.isnull().sum()

Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
Deaths               0
Recovered         2772
dtype: int64

In [7]:
# Fill null with 0 and change Recovered to int
df_final['Recovered'] = df_final['Recovered'].fillna(0)
df_final['Recovered'] = df_final['Recovered'].astype('int')
df_final.isna().sum()

Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
Deaths            0
Recovered         0
dtype: int64

In [8]:
# Check data types
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26136 entries, 0 to 26135
Data columns (total 7 columns):
Country/Region    26136 non-null object
Lat               26136 non-null float64
Long              26136 non-null float64
Date              26136 non-null object
Confirmed         26136 non-null int64
Deaths            26136 non-null int64
Recovered         26136 non-null int32
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 1.5+ MB


In [9]:
# Change date to datetime
df_final.Date=pd.to_datetime(df_final['Date'])

In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26136 entries, 0 to 26135
Data columns (total 7 columns):
Country/Region    26136 non-null object
Lat               26136 non-null float64
Long              26136 non-null float64
Date              26136 non-null datetime64[ns]
Confirmed         26136 non-null int64
Deaths            26136 non-null int64
Recovered         26136 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(1)
memory usage: 1.5+ MB


### Save data on github

In [11]:
# save file
df_final.to_csv('covid_19_complete.csv',index=False)